In [1]:
!pip install transformers 
# # !pip install wget

In [2]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import AutoTokenizer,AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
import random

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np
import re
import os 
os.environ["WANDB_DISABLED"] = "true"

In [3]:

# call the function
df_train = pd.read_excel('train.xlsx')
df_test =  pd.read_excel('test.xlsx')
df_valid = pd.read_excel('valid.xlsx')

df_train['Sentence'] = df_train['Sentence'].apply(lambda x: re.sub('per ','',x))
df_test['Sentence'] = df_test['Sentence'].apply(lambda x: re.sub('per ','',x))
df_valid['Sentence'] = df_valid['Sentence'].apply(lambda x: re.sub('per ','',x))



test_texts = list(df_test['Sentence'])
train_texts = list(df_train['Sentence'])
valid_texts = list(df_valid['Sentence'])

y= LabelEncoder()

train_labels = y.fit_transform(df_train['Emotion'])
valid_labels = y.fit_transform(df_valid['Emotion'])
test_labels = y.fit_transform(df_test['Emotion'])

target_names = list(df_train.Emotion.unique())


In [4]:

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf
        tf.random.set_seed(seed)

set_seed(1)


# check text classification models here: https://huggingface.co/vinai/phobert-base
 
model_name = "vinai/phobert-base" 





# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)


Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)


#see more https://huggingface.co/transformers/v3.4.0/custom_datasets.html
class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset

train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)
test_dataset = NewsGroupsDataset(test_encodings, test_labels)
# load the model and pass to CUDA
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to("cuda")


Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [6]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

training_args = TrainingArguments(
    output_dir='my_model',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=15,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay

)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    tokenizer=tokenizer,                  
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
# train the model
trainer.train()
# evaluate the current model after training
trainer.evaluate()
# saving the fine tuned model & tokenizer

# model_path = "..output/vinai/phobert-base"
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5548
  Num Epochs = 5
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 1850


Step,Training Loss
500,1.525800
1000,0.933600
1500,0.530600


Saving model checkpoint to my_model/checkpoint-500
Configuration saved in my_model/checkpoint-500/config.json
Model weights saved in my_model/checkpoint-500/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-500/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-500/special_tokens_map.json
added tokens file saved in my_model/checkpoint-500/added_tokens.json
Saving model checkpoint to my_model/checkpoint-1000
Configuration saved in my_model/checkpoint-1000/config.json
Model weights saved in my_model/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in my_model/checkpoint-1000/tokenizer_config.json
Special tokens file saved in my_model/checkpoint-1000/special_tokens_map.json
added tokens file saved in my_model/checkpoint-1000/added_tokens.json
Saving model checkpoint to my_model/checkpoint-1500
Configuration saved in my_model/checkpoint-1500/config.json
Model weights saved in my_model/checkpoint-1500/pytorch_model.bin
tokenizer config fil

{'eval_loss': 1.3851919174194336,
 'eval_accuracy': 0.6166180758017493,
 'eval_runtime': 2.0449,
 'eval_samples_per_second': 335.465,
 'eval_steps_per_second': 22.495,
 'epoch': 5.0}

In [7]:
pred = trainer.predict(test_dataset)

# print(test_labels)
y_pred = np.argmax(pred.predictions, axis=1)
# print(y_pred)
print(classification_report(test_labels,y_pred))

***** Running Prediction *****
  Num examples = 693
  Batch size = 15


              precision    recall  f1-score   support

           0       0.56      0.62      0.59        40
           1       0.62      0.57      0.59       132
           2       0.72      0.72      0.72       193
           3       0.69      0.76      0.72        46
           4       0.58      0.55      0.56       129
           5       0.68      0.75      0.71       116
           6       0.74      0.62      0.68        37

    accuracy                           0.66       693
   macro avg       0.65      0.66      0.65       693
weighted avg       0.66      0.66      0.66       693



In [8]:
trainer.save_model('trainer_bert')
tokenizer.save_pretrained("tokenizer_bert")

Saving model checkpoint to trainer-bert
Configuration saved in trainer-bert/config.json
Model weights saved in trainer-bert/pytorch_model.bin
tokenizer config file saved in trainer-bert/tokenizer_config.json
Special tokens file saved in trainer-bert/special_tokens_map.json
added tokens file saved in trainer-bert/added_tokens.json
tokenizer config file saved in tokenizer-bert/tokenizer_config.json
Special tokens file saved in tokenizer-bert/special_tokens_map.json
added tokens file saved in tokenizer-bert/added_tokens.json


('tokenizer-bert/tokenizer_config.json',
 'tokenizer-bert/special_tokens_map.json',
 'tokenizer-bert/vocab.txt',
 'tokenizer-bert/bpe.codes',
 'tokenizer-bert/added_tokens.json')